# Extração de dados do TMDB (The Movie Database)
---
27/09/2025

Este notebook tem como objetivo construir um dataset de filmes utilizando a API do The Movie Database (TMDB). O processo é realizado em duas etapas interligadas:
1. A primeira etapa é a extração dos dados de **índice** dos filmes a partir do endpoint `/discover/movie`
2. A segunda etapa consiste em uma série de requisições para o endpoint `/movie/{movie_id}`, que retorna os dados detalhados de cada filme

###### Todos os endpoints utilizados nessa extração estão detalhados na [documentação oficial da API do TMDB](https://developer.themoviedb.org/reference/intro/getting-started)

In [1]:
import requests
import json
import time
import pandas as pd
from os import environ as env
from concurrent.futures import ThreadPoolExecutor, as_completed


# API credentials config -----------------------------------------------------------------------------------------------------
API_AUTH_CONFIG = {
    "tmdb_api_token": env.get("TMDB_API_TOKEN") or "tmdb_api_token_default",
    "tmdb_api_key": env.get("TMDB_API_KEY") or "tmdb_api_key_default"
}

# Api rate limit config
MAX_REQUESTS_PER_SECOND = 50
MIN_TIME_PER_REQUEST = 1.0 / MAX_REQUESTS_PER_SECOND

# API URLs config -----------------------------------------------------------------------------------------------------
index_url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page=1&sort_by=popularity.desc"

# Pandas configs
pd.set_option('display.max_colwidth', None)

### Passo 1. Requisição dos dados de índice dos filmes

| Componente | Detalhe | Finalidade |
| :--- | :--- | :--- |
| **Endpoint** | **`/discover/movie`** | Obtém a lista de filmes que se enquadram em critérios amplos (ex: por popularidade, gênero, ano de lançamento). |
| **Filtros** | (Especifique os parâmetros usados, ex: `with_genres`, `sort_by`, `page`) | Define o universo de filmes a ser coletado. |
| **Output Crucial** | **`id`** (Identificador único do filme) | O ID é o dado chave que será utilizado na próxima etapa. |

In [ ]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {API_AUTH_CONFIG['tmdb_api_token']}"
}

# Requests configs
num_total_pages = 30

# Function for single index page request
def fetch_index(page):
    index_base_url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={page}&sort_by=popularity.desc"
    
    params = {
        "api_key": API_AUTH_CONFIG['tmdb_api_key'],
    }
    
    index_response = requests.get(index_base_url, params=params)
    index_string = index_response.content.decode('utf-8')
    index_json = json.loads(index_string)
    
    print(f"Fetched: {index_json['page']}")
    
    return index_json

# Function for parallel index page requests
def fetch_index_parallel(num_total_pages, max_workers=5):
    index_complete_result = []
    pages_to_request = range(1, num_total_pages + 1)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        
        for page in pages_to_request:
            start_time = time.time()
            
            future = executor.submit(fetch_index, page)
            futures[future] = page
            
            time_spent = time.time() - start_time
            sleep_time = MIN_TIME_PER_REQUEST - time_spent
            
            if sleep_time > 0:
                time.sleep(sleep_time)
        
        for future in as_completed(futures):
            try:
                index_complete_result.append(future.result())
            except Exception as e:
                mid = futures[future]
                print(f"Erro ao buscar página de índice {mid}: {e}")
    return index_complete_result

index_page_data = fetch_index_parallel(num_total_pages)
print(f"\nTotal de páginas buscadas: {len(index_page_data)}")

Fetched: 2
Fetched: 1
Fetched: 3
Fetched: 4
Fetched: 5
Fetched: 7
Fetched: 6
Fetched: 8
Fetched: 9
Fetched: 10
Fetched: 11
Fetched: 12
Fetched: 13
Fetched: 14
Fetched: 15
Fetched: 16
Fetched: 17
Fetched: 18
Fetched: 19
Fetched: 20
Fetched: 21
Fetched: 22
Fetched: 24
Fetched: 23
Fetched: 25
Fetched: 27
Fetched: 26
Fetched: 29
Fetched: 28
Fetched: 30

Total de páginas buscadas: 30


In [3]:
# print(f'Total pages: {index_page_data['results']}')
# print(f'Total results: {len(index_page_data)}')
index_results_list = [page['results'] for page in index_page_data]
final_index_list = []
for index_page in index_results_list:
    final_index_list.extend(index_page)
    
print(f"Finalizado extend da lista de indices com {len(final_index_list)} resultados.")

Finalizado extend da lista de indices com 600 resultados.


In [4]:
movie_index_df = pd.DataFrame(final_index_list)

# print(movie_index_df)
print(len(movie_index_df))

# print(movie_index_df.columns)

600


### Passo 2: Detalhamento Individual dos Filmes 

| Componente | Detalhe | Finalidade |
| :--- | :--- | :--- |
| **Endpoint** | **`/movie/{movie_id}`** | Realiza uma consulta específica para cada `id` obtido no Passo 1. |
| **Processo** | O código **itera** sobre a lista de IDs extraída. | Garante que cada filme tenha seus dados detalhados coletados. |
| **Output Final** | Dados detalhados (`budget`, `revenue`, `runtime`, `production_companies`, etc.). | Cria o *dataset* final para a análise. |

In [5]:

# Set current search movie id list from index table response
movie_ids = movie_index_df['id'].tolist()

# Function for single movie request
def fetch_film(movie_id):
    detail_url = f'https://api.themoviedb.org/3/movie/{movie_id}'
    
    params = {
        "api_key": API_AUTH_CONFIG['tmdb_api_key'],
        # "append_to_response": "genres"
    }
    
    movie_detail_response = requests.get(detail_url, params=params)
    movie_detail_string = movie_detail_response.content.decode('utf-8')
    movie_detail_json = json.loads(movie_detail_string)
    
    print(f"Fetched: {movie_detail_json['title']}")
    
    return movie_detail_json

# Function for parallel movie detail requests
def fetch_movies_parallel(movie_ids, max_workers=5):
    movie_detail_results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        
        for movie in movie_ids:
            start_time = time.time()
            
            future = executor.submit(fetch_film, movie)
            futures[future] = movie
            
            time_spent = time.time() - start_time
            sleep_time = MIN_TIME_PER_REQUEST - time_spent
            
            if sleep_time > 0:
                time.sleep(sleep_time)
        
        for future in as_completed(futures):
            try:
                movie_detail_results.append(future.result())
            except Exception as e:
                mid = futures[future]
                print(f"Erro ao buscar filme {mid}: {e}")
    return movie_detail_results

movie_detail_data = fetch_movies_parallel(movie_ids)
print(f"\nTotal de filmes buscados: {len(movie_detail_data)}")

Fetched: Karate Kid: Legends
Fetched: Sinners
Fetched: Holy Cow
Fetched: Sorority
Fetched: Moana 2
Fetched: Deadpool & Wolverine
Fetched: Bring Her Back
Fetched: Stockholm Bloodbath
Fetched: The Long Walk
Fetched: Locos de amor, mi primer amor
Fetched: My Fault: London
Fetched: Thunderbolts*
Fetched: The Smashing Machine
Fetched: The Conjuring: The Devil Made Me Do It
Fetched: Dracula: A Love Tale
Fetched: Donde tú quieras
Fetched: Harry Potter and the Philosopher's Stone
Fetched: I Know What You Did Last Summer
Fetched: Get Fast
Fetched: A Minecraft Movie
Fetched: The Conjuring: Last Rites
Fetched: The Lost Princess
Fetched: The Toxic Avenger Unrated
Fetched: Marco
Fetched: Demon Slayer: Kimetsu no Yaiba Infinity Castle
Fetched: Holy Night: Demon Hunters
Fetched: War of the Worlds
Fetched: Play Dirty
Fetched: Primitive War
Fetched: The Fantastic 4: First Steps
Fetched: Fight Another Day
Fetched: Mantis
Fetched: HIM
Fetched: Prisoner of War
Fetched: The Lost Bus
Fetched: Django Undispu

### Passo 3. Gerar as dataframes finais para a camada bronze

Alguns dos campos retornados são arrays de objetos. Esses irão se tornar dataframes separadas, interligadas pelo ID do filme. Em seguida cada uma dessas dataframes, se tornarão um csv para a camada bronze (raw data).

Os campos identificados como array são:
1. `genres`
2. `production_companies`
3. `production_countries`
4. `spoken_languages`

Nesse contexto, os CSVs de saída dessa extração serão:
1. movies.csv
2. genres.csv
3. production_companies.csv
4. production_countries.csv
5. spoken_languages.csv

A chave extrangeira que une cada um desses artefatos, é o ID do filme. Para que isso seja garantido, será adicionada uma coluna `movie_id` em cada nos demais CSVs (com exceção de movies.csv por ser o artefato "base").

In [6]:
array_fields = ["genres", "production_companies", "production_countries", "spoken_languages"]
regular_keys = [key for key in movie_detail_data[0] if key not in array_fields]

regular_movie_fields = []

for movie in movie_detail_data:
    current_movie = {k: v for k, v in movie.items() if k not in array_fields}
    regular_movie_fields.append(current_movie)
    
movie_detail_df = pd.DataFrame(regular_movie_fields)


In [7]:
array_fields_df = {}

for key in array_fields:
    current_df = pd.json_normalize(
        data=movie_detail_data,
        record_path=key,
        meta=['id'],
        record_prefix=f'{key}_'
    )
    current_df = current_df.rename(columns={'id': 'movie_id', f'{key}_id': 'id'})
    array_fields_df[f'{key}_df'] = current_df
    
    
genres_df = array_fields_df['genres_df']
production_companies_df = array_fields_df['production_companies_df']
production_countries_df = array_fields_df['production_countries_df']
spoken_languages_df = array_fields_df['spoken_languages_df']

### Passo 4. Salvar os CSVs na camada bronze

Nesse momento existem 5 dataframes, que serão salvas em arquivos CSVs na camada bronze da seguinte forma:

|Dataframe|Descrição|Arquivo de saída|
|---|---|---|
|movie_detail_df|Dataframe principal com os dados de detalhes de filmes|movies.csv|
|genres_df|Dataframe com dados de gênero dos filmes|genres.csv|
|production_companies_df|Dataframe com dados de produtoras de filmes|production_companies.csv|
|production_countries_df|Dataframe com dados de países da produção dos filmes|production_countries.csv|
|spoken_languages_df|Dataframe com os idiomas falados para cada filme|spoken_languages.csv|

In [8]:
movie_detail_df.to_csv('bronze/movies.csv', index=False)
genres_df.to_csv('bronze/genres.csv', index=False)
production_companies_df.to_csv('bronze/production_companies.csv', index=False)
production_countries_df.to_csv('bronze/production_countries.csv', index=False)
spoken_languages_df.to_csv('bronze/spoken_languages.csv', index=False)

## 3. Ferramentas e Bibliotecas utilizadas

| Ferramenta | Uso no Notebook |
| :--- | :--- |
| **Python `requests`** | Responsável por todas as chamadas HTTP para a API do TMDB. |
| **JSON** | Manipulação e *parsing* das respostas da API, que são formatadas em JSON. |
| **Pandas** | Estruturação e armazenamento dos dados extraídos em formato DataFrame. |